# Using the image_dehazer module:

Github Link: https://github.com/Utkarsh-Deshmukh/Single-Image-Dehazing-Python

## Using the Module:

### Main and Defined Functions:

In [2]:
# Importing Libraries:
# !pip install image_dehazer

import numpy as np
import image_dehazer
import cv2
import time
# !pip install imutils
import imutils
from imutils.video import VideoStream
import copy
%load_ext autotime

ModuleNotFoundError: No module named 'image_dehazer'

time: 4.8 s (started: 2022-12-15 18:09:49 +05:30)


## Single Image Dehazing Code:

In [6]:
if __name__ == "__main__":
    inputImagePath = r"E:\College Education\DA-IICT\Study\Sem 3\Pratik - Projects\Video Dehazing\Input Images & Videos\Input Images\Online Images\foggy_bench.jpg" #Enter image path
    HazeImg = cv2.imread(inputImagePath)
    HazeCorrectedImg = image_dehazer.remove_haze(HazeImg)	
    # Show Image:
    cv2.imshow('a',HazeCorrectedImg)

    # Saving the Image:
    # cv2.imwrite('/content/enhanced_image.png', HazeCorrectedImg);

    # exit at closing of window
    cv2.waitKey(0) & 0xFF == ord('q')
    cv2.destroyAllWindows()

NameError: name 'image_dehazer' is not defined

## Video Dehazing Code:

In [ ]:
if __name__ == "__main__":
    # Video Input:
    inputVideoPath = r"E:\College Education\DA-IICT\Study\Sem 3\Pratik - Projects\Video Dehazing\Input Images & Videos\Input Videos\Vdo_3.mp4"
    vidObj = cv2.VideoCapture(inputVideoPath)

    # Loop for frames:
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    frame_width = vidObj.get(cv2.CAP_PROP_FRAME_WIDTH)
    frame_height = vidObj.get(cv2.CAP_PROP_FRAME_HEIGHT)
    fps = vidObj.get(cv2.CAP_PROP_FPS)
    frame_nos = vidObj.get(cv2.CAP_PROP_FRAME_COUNT)
    save = cv2.VideoWriter('E:\College Education\DA-IICT\Study\Sem 3\Pratik - Projects\Video Dehazing\Input Images & Videos\Input Videos\output.mp4',fourcc, fps, (int(frame_width), int(frame_height)))

    FI = None
    for frame_idx in range(int(frame_nos+1)):
      ret, I = vidObj.read()
    
      # Skipping Frames:
#       if (frame_idx % 10 == 0) and ret:
#         FI = image_dehazer.remove_haze(I)
#       else:
#         save.write(FI)
      
      # Without Skipping Frames:
      if frame_idx and ret:
        FI = final(I)
        save.write(FI)
      

      # Breaking Loop:
      if cv2.waitKey(1) & 0xFF == ord('q'):
        break

      # frame_idx+=10
    vidObj.release()
    save.release()
    cv2.destroyAllWindows()
    
print('Execution Completed')

## Real-Time Dehazing Code:

In [2]:
vidObj_vs = VideoStream(0).start()
time.sleep(1.0)
start_time = time.time()
while True:
    I_vs = vidObj_vs.read()
    cv2.imshow('Input Sequence',I_vs)
    FI_vs =  image_dehazer.remove_haze(I_vs)
    cv2.imshow('Output Sequence',FI_vs)
    # Breaking Loop:
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
#     vidObj_cv.release()
vidObj_vs.stream.release()
cv2.destroyAllWindows()

NameError: name 'VideoStream' is not defined

### Above Code cannot be used for Real-Time analysis as it's too slow.

## Implementation of Same Code from Scratch:

In [8]:
import numpy as np
# import image_dehazer
import cv2
import time
# !pip install imutils
import imutils
from imutils.video import VideoStream
import copy
%load_ext autotime

ModuleNotFoundError: No module named 'image_dehazer'

time: 31 ms (started: 2022-12-15 18:31:49 +05:30)


In [8]:
def Airlight(HazeImg, AirlightMethod, windowSize):
    if(AirlightMethod.lower() == 'fast'):
        A = []
        if(len(HazeImg.shape) == 3):
            for ch in range(len(HazeImg.shape)):
                kernel = np.ones((windowSize, windowSize), np.uint8)
                minImg = cv2.erode(HazeImg[:, :, ch], kernel)
                A.append(int(minImg.max()))
        else:
            kernel = np.ones((windowSize, windowSize), np.uint8)
            minImg = cv2.erode(HazeImg, kernel)
            A.append(int(minImg.max()))
    return(A)

def BoundCon(HazeImg, A, C0, C1, windowSze):
    if(len(HazeImg.shape) == 3):

        t_b = np.maximum((A[0] - HazeImg[:, :, 0].astype(np.float)) / (A[0] - C0),
                         (HazeImg[:, :, 0].astype(np.float) - A[0]) / (C1 - A[0]))
        t_g = np.maximum((A[1] - HazeImg[:, :, 1].astype(np.float)) / (A[1] - C0),
                         (HazeImg[:, :, 1].astype(np.float) - A[1]) / (C1 - A[1]))
        t_r = np.maximum((A[2] - HazeImg[:, :, 2].astype(np.float)) / (A[2] - C0),
                         (HazeImg[:, :, 2].astype(np.float) - A[2]) / (C1 - A[2]))

        MaxVal = np.maximum(t_b, t_g, t_r)
        transmission = np.minimum(MaxVal, 1)
    else:
        transmission = np.maximum((A[0] - HazeImg.astype(np.float)) / (A[0] - C0),
                         (HazeImg.astype(np.float) - A[0]) / (C1 - A[0]))
        transmission = np.minimum(transmission, 1)

    kernel = np.ones((windowSze, windowSze), np.float)
    transmission = cv2.morphologyEx(transmission, cv2.MORPH_CLOSE, kernel=kernel)
    return(transmission)

def CalTransmission(HazeImg, Transmission, regularize_lambda, sigma):
    rows, cols = Transmission.shape

    KirschFilters = LoadFilterBank()

    # Normalize the filters
    for idx, currentFilter in enumerate(KirschFilters):
        KirschFilters[idx] = KirschFilters[idx] / np.linalg.norm(currentFilter)

    # Calculate Weighting function --> [rows, cols. numFilters] --> One Weighting function for every filter
    WFun = []
    for idx, currentFilter in enumerate(KirschFilters):
        WFun.append(CalculateWeightingFunction(HazeImg, currentFilter, sigma))

    # Precompute the constants that are later needed in the optimization step
    tF = np.fft.fft2(Transmission)
    DS = 0

    for i in range(len(KirschFilters)):
        D = psf2otf(KirschFilters[i], (rows, cols))
        DS = DS + (abs(D) ** 2)

    # Cyclic loop for refining t and u --> Section III in the paper
    beta = 1                    # Start Beta value --> selected from the paper
    beta_max = 2**8             # Selected from the paper --> Section III --> "Scene Transmission Estimation"
    beta_rate = 2*np.sqrt(2)    # Selected from the paper

    while(beta < beta_max):
        gamma = regularize_lambda / beta

        # Fixing t first and solving for u
        DU = 0
        for i in range(len(KirschFilters)):
            dt = circularConvFilt(Transmission, KirschFilters[i])
            u = np.maximum((abs(dt) - (WFun[i] / (len(KirschFilters)*beta))), 0) * np.sign(dt)
            DU = DU + np.fft.fft2(circularConvFilt(u, cv2.flip(KirschFilters[i], -1)))

        # Fixing u and solving t --> Equation 26 in the paper
        # Note: In equation 26, the Numerator is the "DU" calculated in the above part of the code
        # In the equation 26, the Denominator is the DS which was computed as a constant in the above code

        Transmission = np.abs(np.fft.ifft2((gamma * tF + DU) / (gamma + DS)))
        beta = beta * beta_rate
    return(Transmission)

def LoadFilterBank():
    KirschFilters = []
    KirschFilters.append(np.array([[-3, -3, -3],   [-3, 0, 5],   [-3, 5, 5]]))
    KirschFilters.append(np.array([[-3, -3, -3],   [-3, 0, -3],  [5, 5, 5]]))
    KirschFilters.append(np.array([[-3, -3, -3],   [5, 0, -3],   [5, 5, -3]]))
    KirschFilters.append(np.array([[5, -3, -3],    [5, 0, -3],   [5, -3, -3]]))
    KirschFilters.append(np.array([[5, 5, -3],     [5, 0, -3],   [-3, -3, -3]]))
    KirschFilters.append(np.array([[5, 5, 5],      [-3, 0, -3],  [-3, -3, -3]]))
    KirschFilters.append(np.array([[-3, 5, 5],     [-3, 0, 5],   [-3, -3, -3]]))
    KirschFilters.append(np.array([[-3, -3, 5],    [-3, 0, 5],   [-3, -3, 5]]))
    KirschFilters.append(np.array([[-1, -1, -1],   [-1, 8, -1],  [-1, -1, -1]]))
    return(KirschFilters)

def CalculateWeightingFunction(HazeImg, Filter, sigma):

    # Computing the weight function... Eq (17) in the paper

    HazeImageDouble = HazeImg.astype(float) / 255.0
    if(len(HazeImg.shape) == 3):
        Red = HazeImageDouble[:, :, 2]
        d_r = circularConvFilt(Red, Filter)

        Green = HazeImageDouble[:, :, 1]
        d_g = circularConvFilt(Green, Filter)

        Blue = HazeImageDouble[:, :, 0]
        d_b = circularConvFilt(Blue, Filter)

        WFun = np.exp(-((d_r**2) + (d_g**2) + (d_b**2)) / (2 * sigma * sigma))
    else:
        d = circularConvFilt(HazeImageDouble, Filter)
        WFun = np.exp(-((d ** 2) + (d ** 2) + (d ** 2)) / (2 * sigma * sigma))
    return(WFun)

def circularConvFilt(Img, Filter):
    FilterHeight, FilterWidth = Filter.shape
    assert (FilterHeight == FilterWidth), 'Filter must be square in shape --> Height must be same as width'
    assert (FilterHeight % 2 == 1), 'Filter dimension must be a odd number.'

    filterHalsSize = int((FilterHeight - 1)/2)
    rows, cols = Img.shape
    PaddedImg = cv2.copyMakeBorder(Img, filterHalsSize, filterHalsSize, filterHalsSize, filterHalsSize, borderType=cv2.BORDER_WRAP)
    FilteredImg = cv2.filter2D(PaddedImg, -1, Filter)
    Result = FilteredImg[filterHalsSize:rows+filterHalsSize, filterHalsSize:cols+filterHalsSize]

    return(Result)

##################
def psf2otf(psf, shape):
    """
    Convert point-spread function to optical transfer function.
    Compute the Fast Fourier Transform (FFT) of the point-spread
    function (PSF) array and creates the optical transfer function (OTF)
    array that is not influenced by the PSF off-centering.
    By default, the OTF array is the same size as the PSF array.
    To ensure that the OTF is not altered due to PSF off-centering, PSF2OTF
    post-pads the PSF array (down or to the right) with zeros to match
    dimensions specified in OUTSIZE, then circularly shifts the values of
    the PSF array up (or to the left) until the central pixel reaches (1,1)
    position.
    Parameters
    ----------
    psf : `numpy.ndarray`
        PSF array
    shape : int
        Output shape of the OTF array
    Returns
    -------
    otf : `numpy.ndarray`
        OTF array
    Notes
    -----
    Adapted from MATLAB psf2otf function
    """
    if np.all(psf == 0):
        return np.zeros_like(psf)

    inshape = psf.shape
    # Pad the PSF to outsize
    psf = zero_pad(psf, shape, position='corner')

    # Circularly shift OTF so that the 'center' of the PSF is
    # [0,0] element of the array
    for axis, axis_size in enumerate(inshape):
        psf = np.roll(psf, -int(axis_size / 2), axis=axis)

    # Compute the OTF
    otf = np.fft.fft2(psf)

    # Estimate the rough number of operations involved in the FFT
    # and discard the PSF imaginary part if within roundoff error
    # roundoff error  = machine epsilon = sys.float_info.epsilon
    # or np.finfo().eps
    n_ops = np.sum(psf.size * np.log2(psf.shape))
    otf = np.real_if_close(otf, tol=n_ops)

    return otf

def zero_pad(image, shape, position='corner'):
    """
    Extends image to a certain size with zeros
    Parameters
    ----------
    image: real 2d `numpy.ndarray`
        Input image
    shape: tuple of int
        Desired output shape of the image
    position : str, optional
        The position of the input image in the output one:
            * 'corner'
                top-left corner (default)
            * 'center'
                centered
    Returns
    -------
    padded_img: real `numpy.ndarray`
        The zero-padded image
    """
    shape = np.asarray(shape, dtype=int)
    imshape = np.asarray(image.shape, dtype=int)

    if np.alltrue(imshape == shape):
        return image

    if np.any(shape <= 0):
        raise ValueError("ZERO_PAD: null or negative shape given")

    dshape = shape - imshape
    if np.any(dshape < 0):
        raise ValueError("ZERO_PAD: target size smaller than source one")

    pad_img = np.zeros(shape, dtype=image.dtype)

    idx, idy = np.indices(imshape)

    if position == 'center':
        if np.any(dshape % 2 != 0):
            raise ValueError("ZERO_PAD: source and target shapes "
                             "have different parity.")
        offx, offy = dshape // 2
    else:
        offx, offy = (0, 0)

    pad_img[idx + offx, idy + offy] = image

    return pad_img

def removeHaze(HazeImg, Transmission, A, delta):
    '''
    :param HazeImg: Hazy input image
    :param Transmission: estimated transmission
    :param A: estimated airlight
    :param delta: fineTuning parameter for dehazing --> default = 0.85
    :return: result --> Dehazed image
    '''

    # This function will implement equation(3) in the paper
    # " https://www.cv-foundation.org/openaccess/content_iccv_2013/papers/Meng_Efficient_Image_Dehazing_2013_ICCV_paper.pdf "

    epsilon = 0.0001
    Transmission = pow(np.maximum(abs(Transmission), epsilon), delta)

    HazeCorrectedImage = copy.deepcopy(HazeImg)
    if(len(HazeImg.shape) == 3):
        for ch in range(len(HazeImg.shape)):
            temp = ((HazeImg[:, :, ch].astype(float) - A[ch]) / Transmission) + A[ch]
            temp = np.maximum(np.minimum(temp, 255), 0)
            HazeCorrectedImage[:, :, ch] = temp
    else:
        temp = ((HazeImg.astype(float) - A[0]) / Transmission) + A[0]
        temp = np.maximum(np.minimum(temp, 255), 0)
        HazeCorrectedImage = temp
    return(HazeCorrectedImage)

if __name__ == '__main__':
    
    HazeImg1 = cv2.VideoCapture(0)
#     time.sleep(1.0)
#     start_time = time.time()
    while True:
        ret, HazeImg = HazeImg1.read()
        cv2.imshow('Input Sequence',HazeImg)

    
        # Resize image
        '''
        Channels = cv2.split(HazeImg)
        rows, cols = Channels[0].shape
        HazeImg = cv2.resize(HazeImg, (int(0.4 * cols), int(0.4 * rows)))
        '''

        # Estimate Airlight
        windowSze = 15
        AirlightMethod = 'fast'
        A = Airlight(HazeImg, AirlightMethod, windowSze)

        # Calculate Boundary Constraints
        windowSze = 3
        C0 = 20         # Default value = 20 (as recommended in the paper)
        C1 = 300        # Default value = 300 (as recommended in the paper)
        Transmission = BoundCon(HazeImg, A, C0, C1, windowSze)                  #   Computing the Transmission using equation (7) in the paper

        # Refine estimate of transmission
        regularize_lambda = 1       # Default value = 1 (as recommended in the paper) --> Regularization parameter, the more this  value, the closer to the original patch wise transmission
        sigma = 0.5
        Transmission = CalTransmission(HazeImg, Transmission, regularize_lambda, sigma)     # Using contextual information

        # Perform DeHazing
        HazeCorrectedImg = removeHaze(HazeImg, Transmission, A, 0.85)

        cv2.imshow('Output Sequence',HazeCorrectedImg)
        # Breaking Loop:
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    #     vidObj_cv.release()
    HazeImg1.stream.release()
    # save.release()
    cv2.destroyAllWindows()
    #cv2.imwrite('outputImages/result.jpg', HazeCorrectedImg)

C:\Users\Pratik\AppData\Local\Temp\ipykernel_10024\3428521197.py:18: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  t_b = np.maximum((A[0] - HazeImg[:, :, 0].astype(np.float)) / (A[0] - C0),
C:\Users\Pratik\AppData\Local\Temp\ipykernel_10024\3428521197.py:19: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  (HazeImg[:, :, 0].astype(np.float) 

AttributeError: 'cv2.VideoCapture' object has no attribute 'stream'

In [5]:
def Airlight(HazeImg, AirlightMethod, windowSize):
    if(AirlightMethod.lower() == 'fast'):
        A = []
        if(len(HazeImg.shape) == 3):
            for ch in range(len(HazeImg.shape)):
                kernel = np.ones((windowSize, windowSize), np.uint8)
                minImg = cv2.erode(HazeImg[:, :, ch], kernel)
                A.append(int(minImg.max()))
        else:
            kernel = np.ones((windowSize, windowSize), np.uint8)
            minImg = cv2.erode(HazeImg, kernel)
            A.append(int(minImg.max()))
    return(A)

def BoundCon(HazeImg, A, C0, C1, windowSze):
    if(len(HazeImg.shape) == 3):

        t_b = np.maximum((A[0] - HazeImg[:, :, 0].astype(np.float)) / (A[0] - C0),
                         (HazeImg[:, :, 0].astype(np.float) - A[0]) / (C1 - A[0]))
        t_g = np.maximum((A[1] - HazeImg[:, :, 1].astype(np.float)) / (A[1] - C0),
                         (HazeImg[:, :, 1].astype(np.float) - A[1]) / (C1 - A[1]))
        t_r = np.maximum((A[2] - HazeImg[:, :, 2].astype(np.float)) / (A[2] - C0),
                         (HazeImg[:, :, 2].astype(np.float) - A[2]) / (C1 - A[2]))

        MaxVal = np.maximum(t_b, t_g, t_r)
        transmission = np.minimum(MaxVal, 1)
    else:
        transmission = np.maximum((A[0] - HazeImg.astype(np.float)) / (A[0] - C0),
                         (HazeImg.astype(np.float) - A[0]) / (C1 - A[0]))
        transmission = np.minimum(transmission, 1)

    kernel = np.ones((windowSze, windowSze), np.float)
    transmission = cv2.morphologyEx(transmission, cv2.MORPH_CLOSE, kernel=kernel)
    return(transmission)

def CalTransmission(HazeImg, Transmission, regularize_lambda, sigma):
    rows, cols = Transmission.shape

    KirschFilters = LoadFilterBank()

    # Normalize the filters
    for idx, currentFilter in enumerate(KirschFilters):
        KirschFilters[idx] = KirschFilters[idx] / np.linalg.norm(currentFilter)

    # Calculate Weighting function --> [rows, cols. numFilters] --> One Weighting function for every filter
    WFun = []
    for idx, currentFilter in enumerate(KirschFilters):
        WFun.append(CalculateWeightingFunction(HazeImg, currentFilter, sigma))

    # Precompute the constants that are later needed in the optimization step
    tF = np.fft.fft2(Transmission)
    DS = 0

    for i in range(len(KirschFilters)):
        D = psf2otf(KirschFilters[i], (rows, cols))
        DS = DS + (abs(D) ** 2)

    # Cyclic loop for refining t and u --> Section III in the paper
    beta = 1                    # Start Beta value --> selected from the paper
    beta_max = 2**8             # Selected from the paper --> Section III --> "Scene Transmission Estimation"
    beta_rate = 2*np.sqrt(2)    # Selected from the paper

    while(beta < beta_max):
        gamma = regularize_lambda / beta

        # Fixing t first and solving for u
        DU = 0
        for i in range(len(KirschFilters)):
            dt = circularConvFilt(Transmission, KirschFilters[i])
            u = np.maximum((abs(dt) - (WFun[i] / (len(KirschFilters)*beta))), 0) * np.sign(dt)
            DU = DU + np.fft.fft2(circularConvFilt(u, cv2.flip(KirschFilters[i], -1)))

        # Fixing u and solving t --> Equation 26 in the paper
        # Note: In equation 26, the Numerator is the "DU" calculated in the above part of the code
        # In the equation 26, the Denominator is the DS which was computed as a constant in the above code

        Transmission = np.abs(np.fft.ifft2((gamma * tF + DU) / (gamma + DS)))
        beta = beta * beta_rate
    return(Transmission)

def LoadFilterBank():
    KirschFilters = []
    KirschFilters.append(np.array([[-3, -3, -3],   [-3, 0, 5],   [-3, 5, 5]]))
    KirschFilters.append(np.array([[-3, -3, -3],   [-3, 0, -3],  [5, 5, 5]]))
    KirschFilters.append(np.array([[-3, -3, -3],   [5, 0, -3],   [5, 5, -3]]))
    KirschFilters.append(np.array([[5, -3, -3],    [5, 0, -3],   [5, -3, -3]]))
    KirschFilters.append(np.array([[5, 5, -3],     [5, 0, -3],   [-3, -3, -3]]))
    KirschFilters.append(np.array([[5, 5, 5],      [-3, 0, -3],  [-3, -3, -3]]))
    KirschFilters.append(np.array([[-3, 5, 5],     [-3, 0, 5],   [-3, -3, -3]]))
    KirschFilters.append(np.array([[-3, -3, 5],    [-3, 0, 5],   [-3, -3, 5]]))
    KirschFilters.append(np.array([[-1, -1, -1],   [-1, 8, -1],  [-1, -1, -1]]))
    return(KirschFilters)

def CalculateWeightingFunction(HazeImg, Filter, sigma):

    # Computing the weight function... Eq (17) in the paper

    HazeImageDouble = HazeImg.astype(float) / 255.0
    if(len(HazeImg.shape) == 3):
        Red = HazeImageDouble[:, :, 2]
        d_r = circularConvFilt(Red, Filter)

        Green = HazeImageDouble[:, :, 1]
        d_g = circularConvFilt(Green, Filter)

        Blue = HazeImageDouble[:, :, 0]
        d_b = circularConvFilt(Blue, Filter)

        WFun = np.exp(-((d_r**2) + (d_g**2) + (d_b**2)) / (2 * sigma * sigma))
    else:
        d = circularConvFilt(HazeImageDouble, Filter)
        WFun = np.exp(-((d ** 2) + (d ** 2) + (d ** 2)) / (2 * sigma * sigma))
    return(WFun)

def circularConvFilt(Img, Filter):
    FilterHeight, FilterWidth = Filter.shape
    assert (FilterHeight == FilterWidth), 'Filter must be square in shape --> Height must be same as width'
    assert (FilterHeight % 2 == 1), 'Filter dimension must be a odd number.'

    filterHalsSize = int((FilterHeight - 1)/2)
    rows, cols = Img.shape
    PaddedImg = cv2.copyMakeBorder(Img, filterHalsSize, filterHalsSize, filterHalsSize, filterHalsSize, borderType=cv2.BORDER_WRAP)
    FilteredImg = cv2.filter2D(PaddedImg, -1, Filter)
    Result = FilteredImg[filterHalsSize:rows+filterHalsSize, filterHalsSize:cols+filterHalsSize]

    return(Result)

##################
def psf2otf(psf, shape):
    """
    Convert point-spread function to optical transfer function.
    Compute the Fast Fourier Transform (FFT) of the point-spread
    function (PSF) array and creates the optical transfer function (OTF)
    array that is not influenced by the PSF off-centering.
    By default, the OTF array is the same size as the PSF array.
    To ensure that the OTF is not altered due to PSF off-centering, PSF2OTF
    post-pads the PSF array (down or to the right) with zeros to match
    dimensions specified in OUTSIZE, then circularly shifts the values of
    the PSF array up (or to the left) until the central pixel reaches (1,1)
    position.
    Parameters
    ----------
    psf : `numpy.ndarray`
        PSF array
    shape : int
        Output shape of the OTF array
    Returns
    -------
    otf : `numpy.ndarray`
        OTF array
    Notes
    -----
    Adapted from MATLAB psf2otf function
    """
    if np.all(psf == 0):
        return np.zeros_like(psf)

    inshape = psf.shape
    # Pad the PSF to outsize
    psf = zero_pad(psf, shape, position='corner')

    # Circularly shift OTF so that the 'center' of the PSF is
    # [0,0] element of the array
    for axis, axis_size in enumerate(inshape):
        psf = np.roll(psf, -int(axis_size / 2), axis=axis)

    # Compute the OTF
    otf = np.fft.fft2(psf)

    # Estimate the rough number of operations involved in the FFT
    # and discard the PSF imaginary part if within roundoff error
    # roundoff error  = machine epsilon = sys.float_info.epsilon
    # or np.finfo().eps
    n_ops = np.sum(psf.size * np.log2(psf.shape))
    otf = np.real_if_close(otf, tol=n_ops)

    return otf

def zero_pad(image, shape, position='corner'):
    """
    Extends image to a certain size with zeros
    Parameters
    ----------
    image: real 2d `numpy.ndarray`
        Input image
    shape: tuple of int
        Desired output shape of the image
    position : str, optional
        The position of the input image in the output one:
            * 'corner'
                top-left corner (default)
            * 'center'
                centered
    Returns
    -------
    padded_img: real `numpy.ndarray`
        The zero-padded image
    """
    shape = np.asarray(shape, dtype=int)
    imshape = np.asarray(image.shape, dtype=int)

    if np.alltrue(imshape == shape):
        return image

    if np.any(shape <= 0):
        raise ValueError("ZERO_PAD: null or negative shape given")

    dshape = shape - imshape
    if np.any(dshape < 0):
        raise ValueError("ZERO_PAD: target size smaller than source one")

    pad_img = np.zeros(shape, dtype=image.dtype)

    idx, idy = np.indices(imshape)

    if position == 'center':
        if np.any(dshape % 2 != 0):
            raise ValueError("ZERO_PAD: source and target shapes "
                             "have different parity.")
        offx, offy = dshape // 2
    else:
        offx, offy = (0, 0)

    pad_img[idx + offx, idy + offy] = image

    return pad_img

def removeHaze(HazeImg, Transmission, A, delta):
    '''
    :param HazeImg: Hazy input image
    :param Transmission: estimated transmission
    :param A: estimated airlight
    :param delta: fineTuning parameter for dehazing --> default = 0.85
    :return: result --> Dehazed image
    '''

    # This function will implement equation(3) in the paper
    # " https://www.cv-foundation.org/openaccess/content_iccv_2013/papers/Meng_Efficient_Image_Dehazing_2013_ICCV_paper.pdf "

    epsilon = 0.0001
    Transmission = pow(np.maximum(abs(Transmission), epsilon), delta)

    HazeCorrectedImage = copy.deepcopy(HazeImg)
    if(len(HazeImg.shape) == 3):
        for ch in range(len(HazeImg.shape)):
            temp = ((HazeImg[:, :, ch].astype(float) - A[ch]) / Transmission) + A[ch]
            temp = np.maximum(np.minimum(temp, 255), 0)
            HazeCorrectedImage[:, :, ch] = temp
    else:
        temp = ((HazeImg.astype(float) - A[0]) / Transmission) + A[0]
        temp = np.maximum(np.minimum(temp, 255), 0)
        HazeCorrectedImage = temp
    return(HazeCorrectedImage)

if __name__ == '__main__':
    
    HazeImg = cv2.imread(r"E:\College Education\DA-IICT\Study\Sem 3\Pratik - Projects\Video Dehazing\Input Images & Videos\Input Images\Online Images\foggy_bench.jpg")
    
#     time.sleep(1.0)
#     start_time = time.time()
#     while True:
#         ret, HazeImg = HazeImg1.read()
#         cv2.imshow('Input Sequence',HazeImg)

    
    # Resize image
    
    Channels = cv2.split(HazeImg)
    rows, cols = Channels[0].shape
    HazeImg = cv2.resize(HazeImg, (int(0.4 * cols), int(0.4 * rows)))
    
    # Estimate Airlight
    windowSze = 15
    AirlightMethod = 'fast'
    A = Airlight(HazeImg, AirlightMethod, windowSze)

    # Calculate Boundary Constraints
    windowSze = 3
    C0 = 20         # Default value = 20 (as recommended in the paper)
    C1 = 300        # Default value = 300 (as recommended in the paper)
    Transmission = BoundCon(HazeImg, A, C0, C1, windowSze)                  #   Computing the Transmission using equation (7) in the paper

    # Refine estimate of transmission
    regularize_lambda = 1       # Default value = 1 (as recommended in the paper) --> Regularization parameter, the more this  value, the closer to the original patch wise transmission
    sigma = 0.5
    Transmission = CalTransmission(HazeImg, Transmission, regularize_lambda, sigma)     # Using contextual information

    # Perform DeHazing
    HazeCorrectedImg = removeHaze(HazeImg, Transmission, A, 0.85)

    cv2.imshow('Output Sequence',HazeCorrectedImg)
#         # Breaking Loop:
    cv2.waitKey(1) & 0xFF == ord('q')
#             break
#     #     vidObj_cv.release()
#     HazeImg1.stream.release()
#     # save.release()
    cv2.destroyAllWindows()
    #cv2.imwrite('outputImages/result.jpg', HazeCorrectedImg)

C:\Users\Pratik\AppData\Local\Temp\ipykernel_4440\4284608655.py:18: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  t_b = np.maximum((A[0] - HazeImg[:, :, 0].astype(np.float)) / (A[0] - C0),
C:\Users\Pratik\AppData\Local\Temp\ipykernel_4440\4284608655.py:19: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  (HazeImg[:, :, 0].astype(np.float) - 

time: 500 ms (started: 2022-12-15 18:21:02 +05:30)


### Above Code cannot be used for Real-Time analysis as it's too slow.

# Code 2: This is for Under Water images (not usefull for us)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2 
from cv2.ximgproc import guidedFilter
from skimage.restoration import denoise_tv_chambolle, denoise_bilateral
from skimage.io import imread, imsave

def KNB(mw,K):
    mwg = rgb2gray(mw)
    r = np.size(mwg)
    
    re,co = np.shape(mwg)
    nbh = np.zeros([re,co,3])
    #nbh_t = zeros([r,3])
    nbh_v = np.zeros([K,3])
    cent = mwg[int(np.floor(re/2)), int(np.floor(co/2))]
    dist = np.zeros(r)
    dist = abs(mwg - cent)
    dist_ord = np.sort(np.ravel(dist[:]))
    dist_K = dist_ord[K-1]
    x,y = np.where(dist <= dist_K)
    nbh[x,y,0] = mw[x,y,0]
    nbh[x,y,1] = mw[x,y,1]
    nbh[x,y,2] = mw[x,y,2]
    nbh_t0 = sorted(np.ravel(nbh[:,:,0]),reverse=True)
    nbh_t1 = sorted(np.ravel(nbh[:,:,1]),reverse=True)
    nbh_t2 = sorted(np.ravel(nbh[:,:,2]),reverse=True)
    nbh_v[:,0] = nbh_t0[0:K]
    nbh_v[:,1] = nbh_t1[0:K]
    nbh_v[:,2] = nbh_t2[0:K]
    return nbh_v

def rgb2gray(img):
    img_gray = np.double( (0.2989*img[:,:,0] + 0.5870*img[:,:,0] + 0.1140*img[:,:,0])) # RGB to Gray Conversion
    return img_gray
        
S = 19 # Defines the size of sliding-window (SxS) for Airlight estimation


### Construction of a synthetic image degraded with haze
#im = 255*double( imread("undegraded.png") )
#gt = (255.0 - double( 255*imread('depth_ground_truth.png') ))
#r,c,p = im.shape
#f_c = zeros([r,c,3])
#beta =  0.9 / 255.0# Dispersion coeficient 
#t0 = exp(-beta*gt)
#C = round(0.95*255) # Ambient light
#f_c[:,:,0] = im[:,:,0]*t0 + C*(1-t0) # Hazy image
#f_c[:,:,1] = im[:,:,1]*t0 + C*(1-t0) # Hazy image
#f_c[:,:,2] = im[:,:,2]*t0 + C*(1-t0) # Hazy image
#f_sin = im[:,:,0:3]

## Read the input hazy image from drive
# imgName = 'flores'
f_c = np.double(cv2.imread(r"E:\College Education\DA-IICT\Study\Sem 3\Pratik - Projects\Video Dehazing\Input Images & Videos\Input Images\Online Images\foggy_bench.jpg")) # Read Input Hazy Image
# cv2.imshow('win1',f_c)
####################
# Size of the Input Image
Nr,Nc,Np = f_c.shape


#########################################
# Extention of the input image to avoid edge issues
A1 = np.concatenate((np.flipud(np.fliplr(f_c)), np.flipud(f_c), np.flipud(np.fliplr(f_c))), axis=1)
A2 = np.concatenate((np.fliplr(f_c),            f_c,            np.fliplr(f_c)), axis=1)
A3 = np.concatenate((np.flipud(np.fliplr(f_c)), np.flipud(f_c), np.flipud(np.fliplr(f_c))), axis=1)
f_proc = np.concatenate( (A1,A2,A3) ,axis=0)
f_proc = f_proc[Nr-int((S-1)/2):2*Nr+int((S-1)/2), Nc-int((S-1)/2):2*Nc+int((S-1)/2),:]

A_test = np.zeros([Nr,Nc])
f_mv = np.zeros([S,S])
K = np.floor(2*S*S/3)
for k in range(Nr):
    leyend = 'Estimating Airlight: ' + str(int(100*(k+1)/Nr)) + '%'
    print(leyend)
    for l in range(Nc):
        f_mv = ( f_proc[ k:S+k, l:S+l ] )
        f_max = f_mv.max()
        f_min = f_mv.min()
        #u = np.mean(f_mv[:])
        u = (f_min + f_max) / 2.0
        #v = (1 + np.var(f_mv[:]))
        v = f_max - f_min
        A_test[k,l] = u / (1 + v)
print('Done!')

x0,y0 = np.where( A_test == A_test.max())
A = np.zeros(3)
A[0] = f_c[x0[0], y0[0],0]
A[1] = f_c[x0[0], y0[0],1]
A[2] = f_c[x0[0], y0[0],2]
A_est = 0.2989*A[0] + 0.5870*A[1] + 0.1140*A[2]

t_est = np.zeros([Nr,Nc])
trans = np.zeros([Nr,Nc])

#PAR = [15, 0.01, 1.0, 4.] # Parameters for maguey
PAR = [9, 0.01, 10.0, 4.0] # Parameters for flores
#PAR = [19, 0.6, 0.6, 6.] # Parameters for fuente

S = PAR[0]      # Defines the size of sliding-window SxS
w = PAR[1]      # Minimum allowed value of transmission
j0 = PAR[2]     # Parameter for transmission estimation
Kdiv = PAR[3]   # Parameter for caculation of K in adaptive neighborhoods

y = np.zeros([Nr,Nc,Np])
K = S**2 - int((S**2)/Kdiv)
 
for k in range(Nr):
    leyend = 'Estimating Transmission: ' + str(int(100*(k+1)/Nr)) + '%'
    print(leyend)
    for l in range(Nc):
        f_w = f_proc[ k:S+k, l:S+l, : ]
        f_v = KNB(f_w, K)
        Fmax = f_v.max()
        Fmin = f_v.min()
        range_fv = Fmax - Fmin
        fv_avg = (Fmin+Fmax)/2.0
        if range_fv < w:
            range_fv = w 
        alpha = range_fv/(j0*A_est)
        t_est[k,l] = (A_est - (alpha*fv_avg + (1-alpha)*Fmin)) / (A_est-alpha*fv_avg)
                
        if t_est[k,l] > 1:
            t_est[k,l] = 1
        if t_est[k,l] < w:
            t_est[k,l] = w
print('Done!')

#trans = denoise_tv_chambolle(t_est, weight=1000.0, multichannel=False)
trans = guidedFilter(np.uint8(f_c),np.uint8(255*t_est),30,0.1) / 255.0

y[:,:,0] = (f_c[:,:,0] - A[0]) / trans + A[0]
y[:,:,1] = (f_c[:,:,1] - A[1]) / trans + A[1] 
y[:,:,2] = (f_c[:,:,2] - A[2]) / trans + A[2]
y[:,:,:] = abs(y[:,:,:])
xs,ys,zs = np.where( y > 255 )
y[xs,ys,zs] = 255.0
                    
plt.subplot(121), plt.imshow(f_c/255.), plt.title('Hazy Image')
plt.subplot(122), plt.imshow(y/255.), plt.title('Processed Image')

# imsave('%s_output.png' % (imgName), np.uint8(y))

Estimating Airlight: 0%
Estimating Airlight: 0%
Estimating Airlight: 0%
Estimating Airlight: 0%
Estimating Airlight: 0%
Estimating Airlight: 1%
Estimating Airlight: 1%
Estimating Airlight: 1%
Estimating Airlight: 1%
Estimating Airlight: 1%
Estimating Airlight: 1%
Estimating Airlight: 2%
Estimating Airlight: 2%
Estimating Airlight: 2%
Estimating Airlight: 2%
Estimating Airlight: 2%
Estimating Airlight: 2%
Estimating Airlight: 3%
Estimating Airlight: 3%
Estimating Airlight: 3%
Estimating Airlight: 3%
Estimating Airlight: 3%
Estimating Airlight: 3%
Estimating Airlight: 4%
Estimating Airlight: 4%
Estimating Airlight: 4%
Estimating Airlight: 4%
Estimating Airlight: 4%
Estimating Airlight: 4%
Estimating Airlight: 5%
Estimating Airlight: 5%
Estimating Airlight: 5%
Estimating Airlight: 5%
Estimating Airlight: 5%
Estimating Airlight: 5%
Estimating Airlight: 6%
Estimating Airlight: 6%
Estimating Airlight: 6%
Estimating Airlight: 6%
Estimating Airlight: 6%
Estimating Airlight: 6%
Estimating Airli

Estimating Airlight: 56%
Estimating Airlight: 56%
Estimating Airlight: 57%
Estimating Airlight: 57%
Estimating Airlight: 57%
Estimating Airlight: 57%
Estimating Airlight: 57%
Estimating Airlight: 57%
Estimating Airlight: 58%
Estimating Airlight: 58%
Estimating Airlight: 58%
Estimating Airlight: 58%
Estimating Airlight: 58%
Estimating Airlight: 58%
Estimating Airlight: 59%
Estimating Airlight: 59%
Estimating Airlight: 59%
Estimating Airlight: 59%
Estimating Airlight: 59%
Estimating Airlight: 59%
Estimating Airlight: 60%
Estimating Airlight: 60%
Estimating Airlight: 60%
Estimating Airlight: 60%
Estimating Airlight: 60%
Estimating Airlight: 60%
Estimating Airlight: 61%
Estimating Airlight: 61%
Estimating Airlight: 61%
Estimating Airlight: 61%
Estimating Airlight: 61%
Estimating Airlight: 61%
Estimating Airlight: 62%
Estimating Airlight: 62%
Estimating Airlight: 62%
Estimating Airlight: 62%
Estimating Airlight: 62%
Estimating Airlight: 62%
Estimating Airlight: 63%
Estimating Airlight: 63%


Estimating Transmission: 10%
Estimating Transmission: 10%
Estimating Transmission: 10%
Estimating Transmission: 10%
Estimating Transmission: 10%
Estimating Transmission: 11%
Estimating Transmission: 11%
Estimating Transmission: 11%
Estimating Transmission: 11%
Estimating Transmission: 11%
Estimating Transmission: 11%
Estimating Transmission: 12%
Estimating Transmission: 12%
Estimating Transmission: 12%
Estimating Transmission: 12%
Estimating Transmission: 12%
Estimating Transmission: 12%
Estimating Transmission: 13%
Estimating Transmission: 13%
Estimating Transmission: 13%
Estimating Transmission: 13%
Estimating Transmission: 13%
Estimating Transmission: 13%
Estimating Transmission: 14%
Estimating Transmission: 14%
Estimating Transmission: 14%
Estimating Transmission: 14%
Estimating Transmission: 14%
Estimating Transmission: 14%
Estimating Transmission: 15%
Estimating Transmission: 15%
Estimating Transmission: 15%
Estimating Transmission: 15%
Estimating Transmission: 15%
Estimating Tra

Estimating Transmission: 57%
Estimating Transmission: 57%
Estimating Transmission: 57%
Estimating Transmission: 57%
Estimating Transmission: 58%
Estimating Transmission: 58%
Estimating Transmission: 58%
Estimating Transmission: 58%
Estimating Transmission: 58%
Estimating Transmission: 58%
Estimating Transmission: 59%
Estimating Transmission: 59%
Estimating Transmission: 59%
Estimating Transmission: 59%
Estimating Transmission: 59%
Estimating Transmission: 59%
Estimating Transmission: 60%
Estimating Transmission: 60%
Estimating Transmission: 60%
Estimating Transmission: 60%
Estimating Transmission: 60%
Estimating Transmission: 60%
Estimating Transmission: 61%
Estimating Transmission: 61%
Estimating Transmission: 61%
Estimating Transmission: 61%
Estimating Transmission: 61%
Estimating Transmission: 61%
Estimating Transmission: 62%
Estimating Transmission: 62%
Estimating Transmission: 62%
Estimating Transmission: 62%
Estimating Transmission: 62%
Estimating Transmission: 62%
Estimating Tra